In [1]:
from sampo.generator import SimpleSynthetic, SyntheticGraphType
from random import Random

rand = Random(123)
ss = SimpleSynthetic(rand)

fixed_wg = ss.work_graph(mode=SyntheticGraphType.GENERAL, cluster_counts=10, bottom_border=100, top_border=100)

fixed_wg.vertex_count

106

In [2]:
from sampo.schemas.stochastic_graph import ProbabilisticFollowingStochasticGraphScheme

# construct the stochastic graph scheme
graph_scheme = ProbabilisticFollowingStochasticGraphScheme(rand=rand, wg=fixed_wg)
defect_graphs = []
for i, node in enumerate(fixed_wg.nodes):
    defect = ss.graph_nodes(top_border=20)
    defect_graphs.append(defect)
    
    defect_prob = rand.random() * 0.5
    graph_scheme.add_part(node=node.id, nodes=defect, prob=defect_prob)

perfect_wg = graph_scheme.prepare_graph().to_work_graph()
print(perfect_wg.vertex_count)

Can not find native module; switching to default
339


In [3]:
graph_scheme.prepare_graph().to_work_graph().vertex_count

338

In [4]:
from sampo.schemas import WorkerProductivityMode, IntervalGaussian
from sampo.schemas.time_estimator import DefaultWorkEstimator, WorkTimeEstimator


def construct_work_estimator(i: int, productivity: WorkerProductivityMode) -> WorkTimeEstimator:
    work_estimator = DefaultWorkEstimator()
    work_estimator.set_productivity_mode(productivity)
    for worker in ['driver', 'fitter', 'manager', 'handyman', 'electrician', 'engineer']:
        work_estimator.set_worker_productivity(IntervalGaussian(0.2 * i + 0.2, 1, 0, 2), worker)
    return work_estimator

work_estimator = construct_work_estimator(5, WorkerProductivityMode.Static)

In [5]:
from sampo.scheduler.heft.prioritization import prioritization

static_prioritization = prioritization(fixed_wg, work_estimator)
static_prioritization

[ac7c2e69-7f2e-eb7e-0352-05d79b683076,
 8ff41e8d-8944-897c-610e-4ad509c47055,
 0d496d4b-da22-594f-573e-8d77550ae889,
 f692ff1b-e245-f7f3-29f2-a2fbf83d58c3,
 dea61187-b7bc-5c51-1072-75716b461015,
 8f9b34c9-5653-d4bd-2293-202c28dfa298,
 ab33e90c-5f42-937f-24dc-25a5add65955,
 29ed479c-3ec3-068c-b3a1-f1315573ae2d,
 d3234519-1410-7b06-be7e-41743c269046,
 1a1f141b-72c5-3633-bb66-ecb350c1eb44,
 6e3e43c2-e941-2995-4ab3-8105056fb885,
 6ea37f12-62e1-0745-96dc-0a90cd620098,
 73211a9a-597b-6ecf-0546-d9e6ab1f5a09,
 737e7bea-ab38-ba7e-b605-d0dffafe0f4e,
 4a8bbf55-ea73-6384-c5bf-60c6a7eb2407,
 7ba35975-cd2e-e163-858c-2e6e57ed22d0,
 8e7bea51-83c4-a23e-844f-cbcfdf13b884,
 17c57804-570f-89e9-91f7-aa69e4b80fdf,
 e497cd2f-65f2-ef76-0336-5a0850b51605,
 3e44ede9-819c-f976-ee08-6a5045917fb4,
 6f33f708-8343-b946-a56a-62c3c4ef7978,
 0e0eaf52-527d-eda5-9afe-de82d1de950d,
 3993539e-5e76-d1e0-b9f0-82f18ba31e52,
 7ea9de52-fde6-2703-49a6-dbeda38873b0,
 96020f97-0d35-ea15-616e-068ff1b8788c,
 b4abf550-49e8-ea0d-f206-

In [6]:
stochastic_wg = graph_scheme.prepare_graph().to_work_graph()
stochastic_wg.vertex_count

327

In [10]:
from sampo.scheduler.heft.prioritization import prioritization_nodes
from sampo.schemas import WorkGraph, GraphNode


def stochastic_prioritization_function(wg: WorkGraph, work_estimator: WorkTimeEstimator) -> list[GraphNode]:
    # wg is stochastic-generated graph
    # here we can use only information from static graph
    # let's use static prioritization

    # TODO Still no followers saved... find problem please
    order = []
    for original_static_node in static_prioritization:
        static_node = wg[original_static_node.id]
        order.append(static_node)
        # haha, get all defects...
        children = static_node.get_following_nodes()
        children_prioritization = prioritization_nodes(children, work_estimator)
        order.extend(children_prioritization)
    return order

In [11]:
from sampo.generator.environment import get_contractor_by_wg
from sampo.pipeline import SchedulingPipeline
from sampo.scheduler import HEFTScheduler

scheduler = HEFTScheduler(prioritization_f=stochastic_prioritization_function, work_estimator=work_estimator)
contractors = [get_contractor_by_wg(stochastic_wg)]

project = SchedulingPipeline.create() \
    .wg(stochastic_wg) \
    .contractors(contractors) \
    .work_estimator(work_estimator) \
    .schedule(scheduler) \
    .finish()[0]

project.schedule.execution_time

295

In [9]:
project.schedule.full_schedule_df

,idx,task_id,task_name,task_name_mapped,contractor,cost,volume,measurement,start,finish,duration,workers,scheduled_work_object
0,0,44faa95b-0bef-5bcf-32bf-e8fa2a2e562e,finish of project,finish of project,6856975b-c6a4-45bc-b26e-7a9a12af32ed,0,0.000000,unit,0,0,0,{},ScheduledWork[work_unit=44faa95b-0bef-5bcf-32b...
1,1,8ff41e8d-8944-897c-610e-4ad509c47055,minimal road,minimal road,6856975b-c6a4-45bc-b26e-7a9a12af32ed,2430,14.011989,km,0,1,1,"{'driver': 67, 'manager': 41, 'handyman': 135}",ScheduledWork[work_unit=8ff41e8d-8944-897c-610...
2,2,ac7c2e69-7f2e-eb7e-0352-05d79b683076,start of project,start of project,6856975b-c6a4-45bc-b26e-7a9a12af32ed,0,0.000000,unit,0,0,0,{},ScheduledWork[work_unit=ac7c2e69-7f2e-eb7e-035...
3,3,0d496d4b-da22-594f-573e-8d77550ae889,temporary road,temporary road,6856975b-c6a4-45bc-b26e-7a9a12af32ed,2430,14.011989,km,1,2,1,"{'driver': 67, 'manager': 41, 'handyman': 135}",ScheduledWork[work_unit=0d496d4b-da22-594f-573...
4,4,f692ff1b-e245-f7f3-29f2-a2fbf83d58c3,minimal road,minimal road,6856975b-c6a4-45bc-b26e-7a9a12af32ed,2350,11.391279,km,2,3,1,"{'driver': 67, 'manager': 33, 'handyman': 135}",ScheduledWork[work_unit=f692ff1b-e245-f7f3-29f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,101,f5e77d4d-cf30-cf2d-9097-a70286ee6241,block dosage inhibitor,block dosage inhibitor,6856975b-c6a4-45bc-b26e-7a9a12af32ed,11760,0.000000,unit,215,257,42,"{'driver': 5, 'fitter': 8, 'manager': 1, 'hand...",ScheduledWork[work_unit=f5e77d4d-cf30-cf2d-909...
102,102,1ad2b334-8008-0777-57f4-42d5232ac74d,block local automation,block local automation,6856975b-c6a4-45bc-b26e-7a9a12af32ed,5850,0.000000,unit,215,254,39,"{'driver': 2, 'fitter': 4, 'manager': 1, 'hand...",ScheduledWork[work_unit=1ad2b334-8008-0777-57f...
103,103,a3858cde-9be1-092e-d23e-e3c934f4f040,block local automation,block local automation,6856975b-c6a4-45bc-b26e-7a9a12af32ed,5850,0.000000,unit,254,293,39,"{'driver': 2, 'fitter': 4, 'manager': 1, 'hand...",ScheduledWork[work_unit=a3858cde-9be1-092e-d23...
104,104,07758380-677a-20a7-a295-7691b1b77a92,cluster handing,cluster handing,6856975b-c6a4-45bc-b26e-7a9a12af32ed,80,0.860000,unit,293,294,1,"{'driver': 2, 'manager': 2, 'engineer': 4}",ScheduledWork[work_unit=07758380-677a-20a7-a29...


In [ ]:
# [node for lst in self._node2followers.values() for lst2, _ in lst for node in lst2 if node.work_unit.name == 'finish of project']
# [(g1, g2) for g1 in defect_graphs for g2 in defect_graphs if len(set(g1).intersection(set(g2))) > 0]
# sorted([node for lst in self._node2followers.values() for lst2, _ in lst for node in lst2], key=lambda x: x.id)
# sorted([(node, prob) for lst in self._node2followers.values() for lst2, prob in lst for node in lst2], key=lambda x: x[0].id)